In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

In [ ]:
import os
import json
import torch
import numpy as np

In [ ]:
x = torch.rand(1, 2, 3)
i = torch.argmax(x, dim=-1)
y = torch.eye(3, 3)
y[i], i

In [ ]:
def bitsum(x_vector, y_vector):
    extra = 0
    z_vector = []
    for x, y in zip(x_vector.flip(0), y_vector.flip(0)):
        bit = x ^ y ^ extra
        extra = x & y | y & extra | extra & x
        z_vector.append(bit)
    z_vector.append(extra)
    z_vector = torch.tensor(z_vector).flip(0)
    return z_vector

# for _ in range(100):
#     x = torch.randint(0, 2, (d,)) #zeros(10).long()
#     y = torch.randint(0, 2, (d,)) #torch.zeros(10).long()
#     z = bitsum(x, y)
#     assert z @ base == x @ base[1:] + y @ base[1:]

def grid(d, budget):
    x = torch.zeros(d).long()
    step = torch.zeros(d).long()
    n = d - int(np.log2(budget - 1))
    step[-n-1] = 1

    X = [x]
    stop = False
    while not stop:
        x = bitsum(x, step)
        stop = x[0] == 1
        x = x[1:]
        X.append(x)

    X[-1] = torch.ones(d).long()
    X = torch.vstack(X)
    return X

d = 10
budget = 17
base = torch.pow(2, torch.arange(0, d + 1).flip(0))
X = grid(d, budget)
print(X @ base[1:], len(X))

In [ ]:
from problems import QUBO
from solvers import BRUTE

In [ ]:
problem = QUBO(d=10, n=2, seed=0)
solver = BRUTE(problem=problem, budget=256, seed=0)
logs = solver.optimize()
logs

In [ ]:
from scripts.create_problem import load_problem_set

In [ ]:
problems = load_problem_set('data/test/Normal(0, 1)/test')
for problem in problems:
    print(problem.info)

In [ ]:
name = "trajectory.pt"
size = os.path.getsize(name) / 1024
print(size)

In [ ]:
trajectory = {
    "x_list": torch.randint(0, 2, size=(100, 50)),
    "y_list": torch.randn(100),
    "m_list": torch.arange(100),
    "time": 0.11
}
name = "trajectory.pt"
torch.save(trajectory, name)
size = os.path.getsize(name) / 1024
print(size)

# loaded_trajectory = torch.load(name, weights_only=True)

In [ ]:
# trajectory_new = {}
# trajectory_new["x_list"] = [x.tolist() for x in trajectory["x_list"]]
# trajectory_new["y_list"] = [y.item() for y in trajectory["y_list"]]
# trajectory_new["m_list"] = [m.item() for m in trajectory["m_list"]]
# trajectory_new
name = "trajectory.json"
with open(name, "w") as f:
    json.dump(trajectory_new, f, indent=4)
size = os.path.getsize(name) / 1024
print(size)

In [ ]:
from dpt.data import OnlineDataset
from scripts.create_problem import load_problem_set

from dpt.data import *
from train_dpt import *


problems = load_problem_set('data/normal/Normal(0, 1)/test')
dataset = OfflineDataset(problems, action='bitflip')

collate_fn = partial(custom_collate_fn, problem_class=getattr(pbs, 'Problem'))
dataloader = DataLoader(dataset=dataset, batch_size=512, collate_fn=collate_fn, num_workers=32)

for batch in dataloader:
    len(batch)

batch = next(iter(dataloader))
# %timeit batch = next(iter(dataloader))

for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

In [ ]:
batch = next(iter(dataloader))
for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

In [ ]:
torch.rand(1, 2, 3)[:, 0:0, :]

In [ ]:
from dpt.train import DPTSolver


DPTSolver(config=load_config('configs/config.yaml')).cuda().device